In [1]:
# Simple, self-contained BERT fine-tune using Hugging Face Trainer.

from pathlib import Path
import os, math, numpy as np, pandas as pd, polars as pl
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, set_seed
)
from transformers.trainer import TrainerCallback

# project folders
candidates = [Path.cwd(), *Path.cwd().parents]
project_root = next((p for p in candidates if (p / "src").exists() and (p / "data").exists()), None)
if project_root is None:
    raise RuntimeError(f"could not find project root from {Path.cwd()}")

DATA = project_root / "data"
INTERIM = DATA / "interim"
PROCESSED = DATA / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

print("project_root:", project_root)
print("device:", "cuda" if torch.cuda.is_available() else "cpu")

# reproducibility
set_seed(42)

/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


project_root: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation
device: cpu


In [2]:
# Read leakage-safe splits
spl = pl.read_parquet(INTERIM / "splits_no_leak.parquet")  # product_text, label, split
assert {"product_text","label","split"} <= set(spl.columns), spl.columns

# Convert to Pandas for Trainer
def to_xy(df_pl):
    pdf = df_pl.to_pandas()
    return pdf["product_text"].astype(str).tolist(), pdf["label"].astype(int).tolist()

X_tr, y_tr = to_xy(spl.filter(pl.col("split")=="train").select("product_text","label"))
X_val, y_val = to_xy(spl.filter(pl.col("split")=="val").select("product_text","label"))
X_te,  y_te  = to_xy(spl.filter(pl.col("split")=="test").select("product_text","label"))

print("sizes -> train:", len(y_tr), " val:", len(y_val), " test:", len(y_te))

# Class weights from train split (inverse frequency)
neg = (np.array(y_tr)==0).sum()
pos = (np.array(y_tr)==1).sum()
w_neg = len(y_tr)/(2*neg)
w_pos = len(y_tr)/(2*pos)
class_weights = torch.tensor([w_neg, w_pos], dtype=torch.float)
print("class weights [neg, pos]:", class_weights.tolist())

# Tokenizer
MODEL_NAME = "bert-base-uncased"
MAX_LEN = 64  # from your preflight check
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Build tokenized datasets
def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_enc = tokenize_texts(X_tr)
val_enc   = tokenize_texts(X_val)
test_enc  = tokenize_texts(X_te)

class SimpleDS(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return self.labels.shape[0]
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

train_ds = SimpleDS(train_enc, y_tr)
val_ds   = SimpleDS(val_enc, y_val)
test_ds  = SimpleDS(test_enc, y_te)

sizes -> train: 27179  val: 6851  test: 763
class weights [neg, pos]: [3.1728928089141846, 0.593531608581543]


In [3]:
# Model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Weighted loss (handles class imbalance)
# We override Trainer.compute_loss to apply CrossEntropy with weights.
class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = class_weights.to(self.model.device) if class_weights is not None else None
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        if self.class_weights is None:
            loss_fct = torch.nn.CrossEntropyLoss()
        else:
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc  = accuracy_score(labels, preds)
    f1m  = f1_score(labels, preds, average="macro")
    prec = precision_score(labels, preds, pos_label=1, zero_division=0)
    rec  = recall_score(labels, preds,    pos_label=1, zero_division=0)
    return {"accuracy": acc, "macro_f1": f1m, "precision_pos": prec, "recall_pos": rec}

output_dir = PROCESSED / "bert_base_uncased_finetuned"
args = TrainingArguments(
    output_dir=str(output_dir),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_ratio=0.06,
    logging_steps=100,
    report_to=[],  # no wandb
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    class_weights=class_weights,
)

train_out = trainer.train()
print("best model path:", trainer.state.best_model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
  0%|          | 0/2550 [00:00<?, ?it/s]le TOKENIZERS_PARALLELISM=(true | false)
/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_

{'loss': 0.4967, 'grad_norm': 2.9535019397735596, 'learning_rate': 1.3071895424836602e-05, 'epoch': 0.12}


  8%|▊         | 200/2550 [01:06<12:17,  3.19it/s]

{'loss': 0.2236, 'grad_norm': 2.592268228530884, 'learning_rate': 1.9607843137254903e-05, 'epoch': 0.24}


 12%|█▏        | 300/2550 [01:38<11:46,  3.19it/s]

...

{'loss': 0.0292, 'grad_norm': 0.06501242518424988, 'learning_rate': 4.1718815185648727e-07, 'epoch': 2.94}


100%|██████████| 2550/2550 [14:06<00:00,  3.78it/s]/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
                                                   
100%|██████████| 2550/2550 [14:25<00:00,  3.78it/s]

{'eval_loss': 0.08361237496137619, 'eval_accuracy': 0.9918260108013429, 'eval_macro_f1': 0.9838865807745745, 'eval_precision_pos': 0.9950265820613959, 'eval_recall_pos': 0.9953679876479671, 'eval_runtime': 18.4362, 'eval_samples_per_second': 371.606, 'eval_steps_per_second': 5.858, 'epoch': 3.0}


100%|██████████| 2550/2550 [14:27<00:00,  2.94it/s]

{'train_runtime': 867.3546, 'train_samples_per_second': 94.007, 'train_steps_per_second': 2.94, 'train_loss': 0.08208271726673724, 'epoch': 3.0}
best model path: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/data/processed/bert_base_uncased_finetuned/checkpoint-2550


In [4]:
# Evaluate on validation and test
val_metrics = trainer.evaluate(val_ds)
print("VAL:", {k: round(v,4) for k,v in val_metrics.items() if isinstance(v,(float,int))})

test_outputs = trainer.predict(test_ds)
test_metrics = {k: (round(v,4) if isinstance(v,(float,int)) else v) for k,v in test_outputs.metrics.items()}
print("TEST:", test_metrics)

# Confusion matrix on test
test_pred = test_outputs.predictions.argmax(axis=-1)
cm = confusion_matrix(y_te, test_pred)
print("\nConfusion matrix [ [TN FP] [FN TP] ]:\n", cm)
print("\nClassification report (test):\n", classification_report(y_te, test_pred, digits=3))

# Save test predictions for manual review
test_csv = PROCESSED / "bert_test_predictions.csv"
pd.DataFrame({
    "product_text": X_te,
    "true_label": y_te,
    "pred_label": test_pred,
    "prob_fashion": torch.softmax(torch.tensor(test_outputs.predictions), dim=1)[:,1].numpy()
}).to_csv(test_csv, index=False)
print("wrote:", test_csv)

/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 108/108 [00:18<00:00,  5.92it/s]
/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


VAL: {'eval_loss': 0.0836, 'eval_accuracy': 0.9918, 'eval_macro_f1': 0.9839, 'eval_precision_pos': 0.995, 'eval_recall_pos': 0.9954, 'eval_runtime': 18.856, 'eval_samples_per_second': 363.333, 'eval_steps_per_second': 5.728, 'epoch': 3.0}


100%|██████████| 12/12 [00:01<00:00,  6.31it/s]


TEST: {'test_loss': 0.0003, 'test_accuracy': 1.0, 'test_macro_f1': 1.0, 'test_precision_pos': 1.0, 'test_recall_pos': 1.0, 'test_runtime': 2.074, 'test_samples_per_second': 367.886, 'test_steps_per_second': 5.786}

Confusion matrix [ [TN FP] [FN TP] ]:
 [[ 94   0]
 [  0 669]]

Classification report (test):
               precision    recall  f1-score   support

           0      1.000     1.000     1.000        94
           1      1.000     1.000     1.000       669

    accuracy                          1.000       763
   macro avg      1.000     1.000     1.000       763
weighted avg      1.000     1.000     1.000       763

wrote: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/data/processed/bert_test_predictions.csv


In [5]:
# Optional: score the latest month’s unlabeled names
events_all = pl.read_parquet(INTERIM / "events_all.parquet")          # all cleaned names
labeled_all = pl.read_parquet(INTERIM / "labeled_all_clean.parquet")  # labeled set

unlabeled = events_all.join(labeled_all.select("product_text").unique(),
                            on="product_text", how="anti").unique()

# pick latest month (max yyyymm)
latest = unlabeled.select("yyyymm").max().item()
ul_latest = unlabeled.filter(pl.col("yyyymm")==latest)

if ul_latest.height > 0:
    ul_texts = ul_latest["product_text"].to_list()
    ul_enc = tokenizer(ul_texts, padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
    ul_ds = SimpleDS(ul_enc, [0]*len(ul_texts))  # dummy labels
    ul_pred = trainer.predict(ul_ds)
    probs = torch.softmax(torch.tensor(ul_pred.predictions), dim=1)[:,1].numpy()
    preds = probs >= 0.5

    out_csv = PROCESSED / f"bert_preds_{latest}.csv"
    pd.DataFrame({
        "month": latest, "product_text": ul_texts,
        "prob_fashion": probs, "pred_label": preds.astype(int)
    }).to_csv(out_csv, index=False)
    print("wrote:", out_csv, "| rows:", len(ul_texts))
else:
    print("no unlabeled rows found for latest month.")

/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 1216/1216 [03:30<00:00,  5.78it/s]


wrote: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/data/processed/bert_preds_202502.csv | rows: 77786
